In [1]:
# import packages
import polars as pl
import numpy as np 
import pandas as pd 

import lightgbm as lgb
import multiprocessing as mp

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder, StandardScaler

from category_encoders.binary import BinaryEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline 

# import warnings
# warnings.filterwarnings('ignore')

In [2]:
prepared_df = pl.read_parquet("../clean-datasets/transactions_prepared.parquet")

prepared_df

user_id,card_index,trx_amount,trx_method,merch_name,merch_city,merch_state,zip_code,merch_category_code,error_status,is_fraud,timestamp_transaction,weekday,hour,minute
i64,i64,f64,cat,str,str,str,str,cat,cat,str,datetime[μs],i8,i8,i8
0,0,134.09,"""Swipe Transact…","""35272132461278…","""La Verne""","""CA""","""9175""","""5300""","""No Error""","""No""",2002-09-01 06:21:00,7,6,21
0,0,38.48,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-01 06:42:00,7,6,42
0,0,120.34,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-02 06:22:00,1,6,22
0,0,128.95,"""Swipe Transact…","""34145274595791…","""Monterey Park""","""CA""","""91754""","""5651""","""No Error""","""No""",2002-09-02 17:45:00,1,17,45
0,0,104.71,"""Swipe Transact…","""58172184461787…","""La Verne""","""CA""","""9175""","""5912""","""No Error""","""No""",2002-09-03 06:23:00,2,6,23
0,0,86.19,"""Swipe Transact…","""-7146670748125…","""Monterey Park""","""CA""","""91755""","""5970""","""No Error""","""No""",2002-09-03 13:53:00,2,13,53
0,0,93.84,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-04 05:51:00,3,5,51
0,0,123.5,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-04 06:09:00,3,6,9
0,0,61.72,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-05 06:14:00,4,6,14
